In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
'''
FEATURES:
Overall action count/ratio
    - 
Overall day count
Monthly action count/ratio
    - Per brand, number of actions in a month / total actions (across all instances) in that month
Penetration (Popularity / Buys)
    - Number of Buys
    - Related Brand Popularity: Among the brands, split into tiers of high popularity vs low popularity
Monthly Aggregation
    - Per brand, number of actions in a month
    - Per brand, average action_type in a month
    - Std. deviation for number of clicks 
    - Per brand, action count by gender in a month
Double 11 Features
Latest One-Week
Repeat Buyer Features
Age Related
Gender Related
'''

'\nFEATURES:\nOverall action count/ratio\n    - \nOverall day count\nMonthly action count/ratio\n    - Per brand, number of actions in a month / total actions (across all instances) in that month\nPenetration (Popularity / Buys)\n    - Number of Buys\n    - Related Brand Popularity: Among the brands, split into tiers of high popularity vs low popularity\nMonthly Aggregation\n    - Per brand, number of actions in a month\n    - Per brand, average action_type in a month\n    - Std. deviation for number of clicks \n    - Per brand, action count by gender in a month\nDouble 11 Features\nLatest One-Week\nRepeat Buyer Features\nAge Related\nGender Related\n'

In [2]:
df = pd.read_csv("./use_data/expanded_training.csv")
df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0
3,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
4,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
380,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0
381,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
382,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
383,95362,253,962,3263,626.0,1111,0,0.0,0.0,0


In [3]:
df_brand = df[['brand_id']]
df_brand

,brand_id
0,3462.0
1,3462.0
2,3462.0
3,3462.0
4,3462.0
...,...
380,247.0
381,247.0
382,247.0
383,626.0


In [4]:
# Unique Brand Count
unique_brand_count = len(pd.unique(df_brand['brand_id']))
print(f'Unique Brand Count: {unique_brand_count}')

brand_occurrences = df_brand['brand_id'].value_counts()
print(brand_occurrences)

Unique Brand Count: 27
247.0     105
683.0      85
3462.0     36
1246.0     30
6208.0     21
5380.0     19
626.0      18
7924.0     12
4631.0      9
2276.0      7
2350.0      6
5946.0      6
7892.0      5
5491.0      4
7936.0      4
1097.0      3
1446.0      2
6230.0      2
5738.0      2
1905.0      2
3931.0      1
777.0       1
7989.0      1
3654.0      1
6590.0      1
8040.0      1
7371.0      1
Name: brand_id, dtype: int64


In [5]:
df['month'] = df['time_stamp'] // 100
df['day'] = df['time_stamp'] % 100
df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label,month,day
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0,11,11
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0,11,11
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0,11,11
3,379824,198,656,145,3462.0,1110,0,5.0,1.0,0,11,10
4,379824,198,656,145,3462.0,1110,0,5.0,1.0,0,11,10
...,...,...,...,...,...,...,...,...,...,...,...,...
380,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0,11,9
381,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0,11,8
382,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0,11,8
383,95362,253,962,3263,626.0,1111,0,0.0,0.0,0,11,11


In [6]:
# Split Month Day into Separate Columns
df_brand['month'] = df['time_stamp'] // 100
df_brand['day'] = df['time_stamp'] % 100
df_brand

/tmp/ipykernel_17471/3351616709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brand['month'] = df['time_stamp'] // 100
/tmp/ipykernel_17471/3351616709.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brand['day'] = df['time_stamp'] % 100


,brand_id,month,day
0,3462.0,11,11
1,3462.0,11,11
2,3462.0,11,11
3,3462.0,11,10
4,3462.0,11,10
...,...,...,...
380,247.0,11,9
381,247.0,11,8
382,247.0,11,8
383,626.0,11,11


Aggregation

Common Aggregate Functions:
- Average
- Count
    > Action Count
- Maximum
- Median
- Minimum
- Mode
- Range
- Sum
- StdDeviation
- NaNMean

In [7]:
# Monthly Action Count

In [8]:
# Monthly Brand Action Counts
df_brand['brand_monthly_action_count'] = df_brand.groupby(['brand_id', 'month']).transform('size')
df_brand

/tmp/ipykernel_17471/3622206883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brand['brand_monthly_action_count'] = df_brand.groupby(['brand_id', 'month']).transform('size')


,brand_id,month,day,brand_monthly_action_count
0,3462.0,11,11,36
1,3462.0,11,11,36
2,3462.0,11,11,36
3,3462.0,11,10,36
4,3462.0,11,10,36
...,...,...,...,...
380,247.0,11,9,104
381,247.0,11,8,104
382,247.0,11,8,104
383,626.0,11,11,18


In [9]:
# Total Action Count in said month
df_brand['month_total_action_count'] = df_brand.groupby(['month']).transform('size')
df_brand


/tmp/ipykernel_17471/496355873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brand['month_total_action_count'] = df_brand.groupby(['month']).transform('size')


,brand_id,month,day,brand_monthly_action_count,month_total_action_count
0,3462.0,11,11,36,362
1,3462.0,11,11,36,362
2,3462.0,11,11,36,362
3,3462.0,11,10,36,362
4,3462.0,11,10,36,362
...,...,...,...,...,...
380,247.0,11,9,104,362
381,247.0,11,8,104,362
382,247.0,11,8,104,362
383,626.0,11,11,18,362


In [10]:
# Monthly Action Count / Ratio (Count/Ratio Type)
df_brand['monthly_action_count_ratio'] = df_brand.groupby(['brand_id', 'month'])['brand_monthly_action_count'].transform(lambda x: x/ df_brand['month_total_action_count'])
df_brand


/tmp/ipykernel_17471/4097056477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brand['monthly_action_count_ratio'] = df_brand.groupby(['brand_id', 'month'])['brand_monthly_action_count'].transform(lambda x: x/ df_brand['month_total_action_count'])


,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio
0,3462.0,11,11,36,362,0.099448
1,3462.0,11,11,36,362,0.099448
2,3462.0,11,11,36,362,0.099448
3,3462.0,11,10,36,362,0.099448
4,3462.0,11,10,36,362,0.099448
...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293
381,247.0,11,8,104,362,0.287293
382,247.0,11,8,104,362,0.287293
383,626.0,11,11,18,362,0.049724


In [11]:
# Per Brand Mean Action Type (Treats Action Type not really Discrete Label)
df_brand['monthly_mean_action_type'] = df.groupby(['brand_id', 'month'])['action_type'].transform('mean')
df_brand

/tmp/ipykernel_17471/3112308950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brand['monthly_mean_action_type'] = df.groupby(['brand_id', 'month'])['action_type'].transform('mean')


,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type
0,3462.0,11,11,36,362,0.099448,0.361111
1,3462.0,11,11,36,362,0.099448,0.361111
2,3462.0,11,11,36,362,0.099448,0.361111
3,3462.0,11,10,36,362,0.099448,0.361111
4,3462.0,11,10,36,362,0.099448,0.361111
...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769
381,247.0,11,8,104,362,0.287293,0.355769
382,247.0,11,8,104,362,0.287293,0.355769
383,626.0,11,11,18,362,0.049724,0.111111



# Gender Interaction Count per Brand

In [12]:
# For Other Columns, modify the groupby condition
male_counts = df.groupby(['brand_id'])['gender'].apply(lambda x: (x == 1).sum())
female_counts = df.groupby(['brand_id'])['gender'].apply(lambda x: (x == 0).sum())
unknown_gender_count = df.groupby(['brand_id'])['gender'].apply(lambda x: (x == 2).sum())
gender_total_brand_counts = pd.DataFrame({'brand_male_count': male_counts, 'brand_female_count': female_counts, 'brand_unknown_gender_count': unknown_gender_count})
gender_total_brand_counts


,brand_male_count,brand_female_count,brand_unknown_gender_count
brand_id,,,
247.0,72,33,0
626.0,0,18,0
683.0,40,44,1
777.0,0,1,0
1097.0,0,3,0
1246.0,19,11,0
1446.0,0,2,0
1905.0,0,1,1
2276.0,0,7,0


In [13]:
df_brand = df_brand.join(gender_total_brand_counts, on='brand_id')
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2
...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0


# Monthly Gender Action Count per Brand

In [14]:
monthly_male_counts = df.groupby(['brand_id', 'month'])['gender'].apply(lambda x: (x == 1).sum())
monthly_female_counts = df.groupby(['brand_id', 'month'])['gender'].apply(lambda x: (x == 0).sum())
monthly_unknown_gender_count = df.groupby(['brand_id', 'month'])['gender'].apply(lambda x: (x == 2).sum())
monthly_gender_total_brand_counts = pd.DataFrame({'month_brand_male_count': monthly_male_counts, 'month_brand_female_count': monthly_female_counts, 'month_brand_unknown_gender_count': monthly_unknown_gender_count})
monthly_gender_total_brand_counts

month_brand_male_count  month_brand_female_count  \
brand_id month                                                     
247.0    10                          1                         0   
         11                         71                        33   
626.0    11                          0                        18   
683.0    5                           6                         0   
         6                           2                         1   
         10                          0                         1   
         11                         32                        42   
777.0    11                          0                         1   
1097.0   11                          0                         3   
1246.0   11                         19                        11   
1446.0   5                           0                         1   
         6                           0                         1   
1905.0   5                           0                         1   
         11                          0                         0   
2276.0   11                          0                         7   
2350.0   11                          0                         6   
3462.0   11                          9                        25   
3654.0   11                          1                         0   
3931.0   11                          0                         1   
4631.0   10                          0                         1   
         11                          7                         1   
5380.0   11                          0                        19   
5491.0   10                          0                         1   
         11                          0                         3   
5738.0   11                          2                         0   
5946.0   11                          0                         6   
6208.0   11                         13                         8   
6230.0   11                          2                         0   
6590.0   10                          1                         0   
7371.0   11                          0                         1   
7892.0   8                           5                         0   
7924.0   11                          2                        10   
7936.0   11                          0                         4   
7989.0   6                           0                         1   
8040.0   11                          0                         1   

                month_brand_unknown_gender_count  
brand_id month                                    
247.0    10                                    0  
         11                                    0  
626.0    11                                    0  
683.0    5                                     0  
         6                                     0  
         10                                    0  
         11                                    1  
777.0    11                                    0  
1097.0   11                                    0  
1246.0   11                                    0  
1446.0   5                                     0  
         6                                     0  
1905.0   5                                     0  
         11                                    1  
2276.0   11                                    0  
2350.0   11                                    0  
3462.0   11                                    2  
3654.0   11                                    0  
3931.0   11                                    0  
4631.0   10                                    0  
         11                                    0  
5380.0   11                                    0  
5491.0   10                                    0  
         11                                    0  
5738.0   11                                    0  
5946.0   11                                    0  
6208.0   11                                    0  
6230.0   11         

In [15]:
df_brand = df_brand.join(monthly_gender_total_brand_counts, on=['brand_id', 'month'])
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,month_brand_male_count,month_brand_female_count,month_brand_unknown_gender_count
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,71,33,0
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,0,18,0


# Brand Gender Count Ratio

In [16]:
gender_counts = df_brand[['brand_male_count', 'brand_female_count', 'brand_unknown_gender_count']].sum(axis=1)
male_ratio = df_brand.apply(lambda row: row['brand_male_count'] / gender_counts[row.name], axis=1)
female_ratio = df_brand.apply(lambda row: row['brand_female_count'] / gender_counts[row.name], axis=1)
unknown_gender_ratio = df_brand.apply(lambda row: row['brand_unknown_gender_count'] / gender_counts[row.name], axis=1)
df_brand['brand_male_count_ratio'] = male_ratio
df_brand['brand_female_count_ratio'] = female_ratio
df_brand['brand_unknown_count_ratio'] = unknown_gender_ratio
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,month_brand_male_count,month_brand_female_count,month_brand_unknown_gender_count,brand_male_count_ratio,brand_female_count_ratio,brand_unknown_count_ratio
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,0,18,0,0.000000,1.000000,0.000000


# Brand Monthly Gender Count Ratio

In [17]:
df_brand['brand_monthly_male_count_ratio'] = df_brand['month_brand_male_count'] / df_brand['brand_monthly_action_count']
df_brand['brand_monthly_female_count_ratio'] = df_brand['month_brand_female_count'] / df_brand['brand_monthly_action_count']
df_brand['brand_monthly_unknown_count_ratio'] = df_brand['month_brand_unknown_gender_count'] / df_brand['brand_monthly_action_count']
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,month_brand_male_count,month_brand_female_count,month_brand_unknown_gender_count,brand_male_count_ratio,brand_female_count_ratio,brand_unknown_count_ratio,brand_monthly_male_count_ratio,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,0,18,0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000


# All Above Confirmed

# Penetration

In [18]:
df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label,month,day
0,379824,198,656,145,3462.0,1111,0,5.0,1.0,0,11,11
1,379824,198,656,145,3462.0,1111,0,5.0,1.0,0,11,11
2,379824,198,656,145,3462.0,1111,2,5.0,1.0,0,11,11
3,379824,198,656,145,3462.0,1110,0,5.0,1.0,0,11,10
4,379824,198,656,145,3462.0,1110,0,5.0,1.0,0,11,10
...,...,...,...,...,...,...,...,...,...,...,...,...
380,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0,11,9
381,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0,11,8
382,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0,11,8
383,95362,253,962,3263,626.0,1111,0,0.0,0.0,0,11,11


In [19]:
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,month_brand_male_count,month_brand_female_count,month_brand_unknown_gender_count,brand_male_count_ratio,brand_female_count_ratio,brand_unknown_count_ratio,brand_monthly_male_count_ratio,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,0,18,0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000


# Number of Buys Per Brand

In [20]:
brand_buy = pd.DataFrame({'brand_buys': df.groupby(['brand_id'])['action_type'].apply(lambda x: (x == 2).sum())})
df_brand = df_brand.join(brand_buy, on=['brand_id'])
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,month_brand_male_count,month_brand_female_count,month_brand_unknown_gender_count,brand_male_count_ratio,brand_female_count_ratio,brand_unknown_count_ratio,brand_monthly_male_count_ratio,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio,brand_buys
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,9,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000,14
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000,14
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,71,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000,14
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,0,18,0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1


# Buy Ratio per Brand (Brand Buy Count / Brand Total Actions)

In [21]:
df_brand['brand_buy_ratio'] = df_brand['brand_buys'] / df_brand.groupby(['brand_id']).transform('size')
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,...,month_brand_female_count,month_brand_unknown_gender_count,brand_male_count_ratio,brand_female_count_ratio,brand_unknown_count_ratio,brand_monthly_male_count_ratio,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio,brand_buys,brand_buy_ratio
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,25,2,0.250000,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,...,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000,14,0.133333
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000,14,0.133333
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,33,0,0.685714,0.314286,0.000000,0.682692,0.317308,0.000000,14,0.133333
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,...,18,0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1,0.055556


# Gender-Buy Count per Brand

In [22]:
brand_male_buy = df[df.action_type == 2].groupby(['brand_id'])['gender'].apply(lambda x: (x == 1).sum()) 
brand_female_buy = df[df.action_type == 2].groupby(['brand_id'])['gender'].apply(lambda x: (x == 0).sum())
brand_unknown_buy = df[df.action_type == 2].groupby(['brand_id'])['gender'].apply(lambda x: (x == 2).sum())
gender_buys = pd.DataFrame({'brand_male_buy_count': brand_male_buy, 'brand_female_buy_count': brand_female_buy, 'brand_unknown_buy_count': brand_unknown_buy})
df_brand = df_brand.join(gender_buys, on=['brand_id'])
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,...,brand_female_count_ratio,brand_unknown_count_ratio,brand_monthly_male_count_ratio,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio,brand_buys,brand_buy_ratio,brand_male_buy_count,brand_female_buy_count,brand_unknown_buy_count
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889,1.0,4.0,0.0
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889,1.0,4.0,0.0
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889,1.0,4.0,0.0
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889,1.0,4.0,0.0
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,0.250000,0.694444,0.055556,5,0.138889,1.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,...,0.314286,0.000000,0.682692,0.317308,0.000000,14,0.133333,8.0,6.0,0.0
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.314286,0.000000,0.682692,0.317308,0.000000,14,0.133333,8.0,6.0,0.0
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.314286,0.000000,0.682692,0.317308,0.000000,14,0.133333,8.0,6.0,0.0
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,...,1.000000,0.000000,0.000000,1.000000,0.000000,1,0.055556,0.0,1.0,0.0


# Gender-Buy Ratio per Brand (Buy Count of a Gender / Total Buy Count for that Brand)

In [23]:
df_brand['brand_male_buy_ratio'] = df_brand['brand_male_buy_count'] / df_brand['brand_buys']
df_brand['brand_female_buy_ratio'] = df_brand['brand_female_buy_count'] / df_brand['brand_buys']
df_brand['brand_unknown_buy_ratio'] = df_brand['brand_unknown_buy_count'] / df_brand['brand_buys']
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,...,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio,brand_buys,brand_buy_ratio,brand_male_buy_count,brand_female_buy_count,brand_unknown_buy_count,brand_male_buy_ratio,brand_female_buy_ratio,brand_unknown_buy_ratio
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,...,0.317308,0.000000,14,0.133333,8.0,6.0,0.0,0.571429,0.428571,0.0
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.317308,0.000000,14,0.133333,8.0,6.0,0.0,0.571429,0.428571,0.0
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.317308,0.000000,14,0.133333,8.0,6.0,0.0,0.571429,0.428571,0.0
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,...,1.000000,0.000000,1,0.055556,0.0,1.0,0.0,0.000000,1.000000,0.0


# Resolving NaNs: When brand_buys is 0, math results in some NaNs for ratios 

In [24]:
# Visualize rows with NaNs
nan_mask = df_brand.isna().any(axis=1)
nan_rows = df_brand[nan_mask]
nan_rows

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,...,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio,brand_buys,brand_buy_ratio,brand_male_buy_count,brand_female_buy_count,brand_unknown_buy_count,brand_male_buy_ratio,brand_female_buy_ratio,brand_unknown_buy_ratio
35,2350.0,11,8,6,362,0.016575,0.0,0,6,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
36,2350.0,11,8,6,362,0.016575,0.0,0,6,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
39,2350.0,11,5,6,362,0.016575,0.0,0,6,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
48,1905.0,5,20,1,8,0.125000,0.0,0,1,1,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
62,5491.0,11,3,3,362,0.008287,0.0,0,4,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
127,777.0,11,11,1,362,0.002762,0.0,0,1,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
139,1097.0,11,10,3,362,0.008287,0.0,0,3,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
143,3931.0,11,11,1,362,0.002762,0.0,0,1,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
165,1446.0,5,28,1,8,0.125000,0.0,0,2,0,...,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
192,7892.0,8,4,5,5,1.000000,0.0,5,0,0,...,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# If Brand_Buys is 0, the previous method generates NaNs. Set these NaNs to 0
condition = df_brand['brand_buys'] == 0
df_brand.loc[condition, ['brand_male_buy_count', 'brand_female_buy_count', 'brand_unknown_buy_count', 'brand_male_buy_ratio', 'brand_female_buy_ratio', 'brand_unknown_buy_ratio']] = 0
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,...,brand_monthly_female_count_ratio,brand_monthly_unknown_count_ratio,brand_buys,brand_buy_ratio,brand_male_buy_count,brand_female_buy_count,brand_unknown_buy_count,brand_male_buy_ratio,brand_female_buy_ratio,brand_unknown_buy_ratio
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.694444,0.055556,5,0.138889,1.0,4.0,0.0,0.200000,0.800000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,...,0.317308,0.000000,14,0.133333,8.0,6.0,0.0,0.571429,0.428571,0.0
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.317308,0.000000,14,0.133333,8.0,6.0,0.0,0.571429,0.428571,0.0
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.317308,0.000000,14,0.133333,8.0,6.0,0.0,0.571429,0.428571,0.0
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,...,1.000000,0.000000,1,0.055556,0.0,1.0,0.0,0.000000,1.000000,0.0


# Check for unaccounted NaNs

In [29]:
print(sum(df_brand.isna().sum()))

0

# Age-Related

# AgeGroup Counts per Brand

In [27]:
# For Other Columns, modify the groupby condition
ageGroup_0_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 0).sum())
ageGroup_1_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 1).sum())
ageGroup_2_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 2).sum())
ageGroup_3_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 3).sum())
ageGroup_4_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 4).sum())
ageGroup_5_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 5).sum())
ageGroup_6_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 6).sum())
ageGroup_7_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 7).sum())
ageGroup_8_counts = df.groupby(['brand_id'])['age_range'].apply(lambda x: (x == 8).sum())
gender_total_brand_counts = pd.DataFrame({'ageGroup_1_counts': ageGroup_1_counts,'ageGroup_2_counts': ageGroup_2_counts,'ageGroup_3_counts': ageGroup_3_counts,'ageGroup_4_counts': ageGroup_4_counts,'ageGroup_5_counts': ageGroup_5_counts,
                                          'ageGroup_6_counts': ageGroup_6_counts,'ageGroup_7_counts': ageGroup_7_counts,'ageGroup_8_counts': ageGroup_8_counts, 'ageGroup_0_counts': ageGroup_0_counts})
df_brand = df_brand.join(gender_total_brand_counts, on='brand_id')
df_brand

,brand_id,month,day,brand_monthly_action_count,month_total_action_count,monthly_action_count_ratio,monthly_mean_action_type,brand_male_count,brand_female_count,brand_unknown_gender_count,...,brand_unknown_buy_ratio,ageGroup_1_counts,ageGroup_2_counts,ageGroup_3_counts,ageGroup_4_counts,ageGroup_5_counts,ageGroup_6_counts,ageGroup_7_counts,ageGroup_8_counts,ageGroup_0_counts
0,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.0,0,2,1,10,7,8,0,0,8
1,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.0,0,2,1,10,7,8,0,0,8
2,3462.0,11,11,36,362,0.099448,0.361111,9,25,2,...,0.0,0,2,1,10,7,8,0,0,8
3,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.0,0,2,1,10,7,8,0,0,8
4,3462.0,11,10,36,362,0.099448,0.361111,9,25,2,...,0.0,0,2,1,10,7,8,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,247.0,11,9,104,362,0.287293,0.355769,72,33,0,...,0.0,0,25,24,35,1,3,0,0,17
381,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.0,0,25,24,35,1,3,0,0,17
382,247.0,11,8,104,362,0.287293,0.355769,72,33,0,...,0.0,0,25,24,35,1,3,0,0,17
383,626.0,11,11,18,362,0.049724,0.111111,0,18,0,...,0.0,0,10,0,6,0,0,0,0,2
